In [ ]:
import numpy
import pandas
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
df = pd.read_csv('labels.csv')
df.head()

In [ ]:
import xml.etree.ElementTree as xet

In [ ]:
filename=df['filepath'][0]
filename

In [ ]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [ ]:
getFilename(filename)

In [ ]:
image_path=list(df['filepath'].apply(getFilename))
image_path

In [ ]:
file_path = image_path[0]
file_path

In [ ]:
img= cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.rectangle(img,(42,94),(205,123),(0,255),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
labels = df.iloc[:,1:].values

In [ ]:
len(image_path)

In [ ]:
data =[]
output=[]
for ind in range (len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    #preprocessing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 #normalization
    #normalization to labels
    xmin,xmax,ymin,ymax= labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax= ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) #normalized output
    #---------------append
    data.append(norm_load_image_arr)
    output.append(label_norm)
    

In [ ]:
x=np.array(data,dtype=np.float32)
y=np.array(data,dtype=np.float32)

In [ ]:
x.shape,y.shape

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
from tensorflow.keras.applications import MobileNetV2, InceptionResNetV2,InceptionV3
from tensorflow.keras.layers import Dense, Dropout, Flatten,Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [26]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=True,
                                   input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
#.......................
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel= Dense(500,activation="relu")(headmodel)
headmodel= Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
#model..................
model= Model (inputs= inception_resnet.input, outputs= headmodel)

In [ ]:
#compile model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()


## model training

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
tfb=TensorBoard("object_detection")

In [24]:
history = model.fit(x=x_train,y=y_train,batch_size=100,epochs=10,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/10


ValueError: in user code:

    File "C:\Users\HP\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\HP\anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\HP\anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\HP\anaconda3\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\HP\anaconda3\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "C:\Users\HP\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\HP\anaconda